## Part 1 - Topic Modeling

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from sparknlp.annotator import StopWordsCleaner
from pyspark.sql import types as T
from pyspark.sql import functions as F
from sparknlp.annotator import NGramGenerator
from sparknlp.base import Finisher
from pyspark.ml.tuning import ParamGridBuilder

spark = sparknlp.start()

data = spark.read.csv("../cleaned_moral_scores.csv", header= True).select(["id", "cleaned_text"])

#Preprocessing
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "wasnt", "yeah", "okay", "lol"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement", "op"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_", "ampxb"]

stopwords = english + time + reddit + topic_specific

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "unigrams")\
      .setOutputCol("pos")

finisher = Finisher().setInputCols(['unigrams', 'pos'])

my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          #ngrammer,
          pos,
          finisher
      ])

pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.persist()

#Filter by POS
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('final') \
     .setStopWords(stopwords)

last_finisher = Finisher() \
     .setInputCols(['final']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)

processed_data.unpersist()
final_data.persist()

## Vectorization
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_final', outputCol='tf_features', minDF=0.01, maxDF=0.80, vocabSize= 2000)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

final_data.unpersist()
tfidf_result.persist()

24/11/28 16:16:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[id: string, cleaned_text: string, finished_unigrams: array<string>, finished_pos: array<string>, filtered_unigrams_by_pos: array<string>, joined_tokens: string, finished_final: array<string>, tf_features: vector, tf_idf_features: vector]

Functions to evaluate and interpret model

In [2]:
vocab = tf_model.vocabulary

def evaluate_model(model, data):
    log_likelihood = model.logLikelihood(data)
    perplexity = model.logPerplexity(data)
    return log_likelihood, perplexity

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [7]:
## LDA Model 
best_lda = LDA(k=9, maxIter=100, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', topicConcentration= 0.04, seed=2503)
best_lda_model = best_lda.fit(tfidf_result)
print(evaluate_model(best_lda_model, tfidf_result))
num_top_words = 15
topics = best_lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

24/11/28 16:40:05 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


(-173160376.01935717, 5.871522756470789)
+-----+---------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                             |
+-----+---------------------------------------------------------------------------------------------------------------------------------------+
|0    |[therapy, emotion, relationship, feeling, therapist, depression, emotional, child, toxic, god, behavior, response, anxiety, sad, angry]|
|1    |[book, game, video, read, value, youtube, opinion, purpose, play, waste, teach, information, knowledgeable, development, boundaries]   |
|2    |[woman, man, social, media, sex, porn, date, relationship, partner, anxiety, guy, addiction, girl, anxious, dude]                      |
|3    |[skill, interest, hobbies, activity, content, language, hobby, attention, specific, inte

Preliminary labels


Topic 0: Mental Health and Emotions.     


Topic 1: Media (Books and entertainment)


Topic 2: Relationships, sexual life and social media


Topic 3: Skills and leisure


Topic 4: Mindset and beliefs 


Topic 5: Personal struggles


Topic 6: Health and Fitness


Topic 7: Confidence


Topic 8: Career and Education

## Part 2. Set topics as attributes 

Create dataframe with topic distributions

In [10]:
df_lda = best_lda_model.transform(tfidf_result)
df_lda.select('id','cleaned_text','topicDistribution').show(5)

+-----+--------------------+--------------------+
|   id|        cleaned_text|   topicDistribution|
+-----+--------------------+--------------------+
|hk5r2|i had an appointm...|[0.23909905329275...|
|iqimz|i created this si...|[0.00256239065072...|
|pfzt5|hello everyone  i...|[0.00137730985685...|
|pk714|i grew up with bo...|[0.17355737574666...|
|q0q8x|i have to ask whe...|[0.15530426629543...|
+-----+--------------------+--------------------+
only showing top 5 rows



Create one column for each topic (asked ChatGPT: how do I create a column for each topic probability after applying pyspark lda?)

In [12]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType

# UDF to convert topicDistribution vector into a list
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

# Apply the UDF to get an array column of topic probabilities
transformed_df = df_lda.withColumn("topic_probs", vector_to_array_udf(col("topicDistribution"))).select('id','topic_probs')

# Create separate columns for each topic probability
for topic_idx in range(9):
    transformed_df = transformed_df.withColumn(f"topic{topic_idx}", col("topic_probs")[topic_idx])

# Drop the intermediate topic_probs column
df_topics_dist = transformed_df.drop("topic_probs")

df_topics_dist.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id|              topic0|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|hk5r2| 0.23909905329275474|0.003794878211208...|0.003861564949996271|0.004758983378604356|0.006398695452876394|0.004991734465360932|  0.7272430358276163|0.004889269891121757|0.004962784530460586|
|iqimz|0.002562390650726633|  0.2805332651672351|0.002023462769260...| 0.49903159473931813| 0.13775133970567327|0.002616334448758515|0.002803259760956...| 0.07007747566426353|0.002600877093808...|
|pfzt5|0.001377

Define what's a good threshold to consider a given topic relevant or not

First, see the distributions for each topic

In [13]:
percentiles_t0 = df_topics_dist.approxQuantile("topic0", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t0

[0.0013696795345640748,
 0.00238743770728943,
 0.004777165909816657,
 0.17345734332859783,
 0.39003229598599376]

In [14]:
percentiles_t1 = df_topics_dist.approxQuantile("topic1", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t1

[0.001019498893924858,
 0.0017498893763533804,
 0.0032317507221397536,
 0.0755017092700805,
 0.2215711278098578]

In [15]:
percentiles_t2 = df_topics_dist.approxQuantile("topic2", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t2

[0.0009906165283924426,
 0.0016843590224861701,
 0.0031350511740660452,
 0.07887485059823934,
 0.27684306568320766]

In [16]:
percentiles_t3 = df_topics_dist.approxQuantile("topic3", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t3

[0.0013677939728476076,
 0.0023717419633680873,
 0.004716963825803371,
 0.1517364235421411,
 0.33377640043947515]

In [17]:
percentiles_t4 = df_topics_dist.approxQuantile("topic4", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t4

[0.0020476194144534434,
 0.003604132973627567,
 0.009464815471161497,
 0.2435928366827867,
 0.4645083773743787]

In [18]:
percentiles_t5 = df_topics_dist.approxQuantile("topic5", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t5

[0.00147049770665158,
 0.0025007582801208107,
 0.005155637315812886,
 0.16701915401399894,
 0.354306647701707]

In [19]:
percentiles_t6 = df_topics_dist.approxQuantile("topic6", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t6

[0.0015398361768300926,
 0.0026789870855917376,
 0.005539304407351832,
 0.19339328158343394,
 0.4759923224848233]

In [20]:
percentiles_t7 = df_topics_dist.approxQuantile("topic7", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t7

[0.0013728213155570336,
 0.002363061084501264,
 0.004773134042861546,
 0.15163453902608298,
 0.3534453332125293]

In [21]:
percentiles_t8 = df_topics_dist.approxQuantile("topic8", [0.1, 0.25, 0.5, 0.75, 0.9], 0.01)
percentiles_t8

[0.0015080232215328203,
 0.00252661527008093,
 0.004999602509471588,
 0.14942827785687648,
 0.3624781111210678]

Intuitively, 0.1 seems like a good threshold to define if a topic is relevant or not. Let's test it with a few documents 

In [22]:
# Merge two dataframes 
df_merged = data.join(df_topics_dist, on="id", how="inner")
df_merged.show(5)

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     id|        cleaned_text|              topic0|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1001497|i am meeting a po...| 0.46694974426533065|8.854661419890824E-4|  0.1914368135291056| 0.13460432393535218|0.001496487331297...|0.001164712366294...|0.001248912091794...| 0.20105548774926266|0.001158052589572546|
|1001uik|hey so basically ...|0.001071859132455...|8.319873616376172E-4|8.466198940519703E-4|0.001043498301123...|  0.18

In [35]:
column_mapping = {
    'topic0': 0,
    'topic1': 1,
    'topic2': 2,
    'topic3': 3,
    'topic4': 4,
    'topic5': 5,
    'topic6': 6,
    'topic7': 7,
    'topic8': 8}

def create_items(input_df):
    # Create a new column 'relevant topics' with the integers representing the topics with a probability greater than 0.2
        df = input_df.withColumn(
        "relevant_topics",
        F.array([
        F.when(F.col(col) > 0.2, F.lit(value)).otherwise(None) 
        for col, value in column_mapping.items()]))

        # Filter out None values from the array
        df = df.withColumn("relevant", F.expr("filter(relevant_topics, x -> x is not null)"))
        
        return df.select('id','cleaned_text','relevant') 
    
    
topics_attr = create_items(df_merged)
topics_attr.show(10, truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Good enough! Save to parquet file for further analysis

In [36]:
topics_attr.write.mode("overwrite").parquet("data_with_topics.parquet")